## Applied Data Science Capstone Week 3 - Segmenting and Clustering Neighborhoods in Toronto

## 4. Submit a link to your Notebook on your Github repository. (10 marks)
### Parse the webpage using Beautiful Soup

In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Scrape the page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Request page
response = requests.get(url)

# Read page content
soup = BeautifulSoup(response.content)

# Get the data from the rows (and skip header)
data_raw = soup.select(".wikitable tr:not(:first-child)")

### Clean the data and setup the dataframe

In [57]:
all_data = {"PostalCode": [], "Borough": [], "Neighborhood": []}
i = 0

# Loop all rows
for row in data_raw:
    tds = row.select("td")
    
    # Get row values
    postalcode = tds[0].get_text().strip()
    borough = tds[1].get_text().strip()
    neighborhood = tds[2].get_text().strip()
    
    # Skip where Borough = "Not assigned"
    if borough == "Not assigned":
        continue
    
    # One row per Postal Code
    if postalcode not in all_data["PostalCode"]:
        all_data["PostalCode"].append(postalcode)
        all_data["Neighborhood"].append(neighborhood)
        
        # One borough per Postal Code
        if borough not in all_data:
            all_data["Borough"].append(borough)
    else:
        # Concatenate the "Neighborhood" values
        index_postalcode = all_data["PostalCode"].index(postalcode)
        all_data["Neighborhood"][index_postalcode] +=  ", " + neighborhood

# Dictionary with the data to pass to a dataframe
data = {
    "PostalCode": all_data["PostalCode"],
    "Borough": all_data["Borough"],
    "Neighborhood": all_data["Neighborhood"]
}

# Setup Dataframe from the dictionary
df = pd.DataFrame(data)

# Replace "Neighborhood" columns having "Not assigned" with the same value as the Borough in the same row
for index, row in df.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]

### Dataframe shape

In [58]:
df.shape

(103, 3)